In [1]:
%matplotlib notebook

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
import time
from scipy.optimize import minimize
from climin.util import optimizer

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [3]:
from schlichtanders.myoptimizers import batch

In [4]:
from theano_models import flatten_parameters, reparameterize_map
from theano_models import InvertibleModel

from theano_models.util import shared, softplus, softplus_inv, total_size

from theano_models.postmaps import scipy_postmap, climin_postmap, numericalize_postmap

import theano_models.probabilistic_models as pm
import theano_models.deterministic_models as dm

ImportError: cannot import name SymbolicSharedVariable

# Deterministic Model - one dimensional

## model

In [ ]:
model = dm.RadialTransform()
model

In [ ]:
model.map('parameters_positive', reparameterize_map(softplus, softplus_inv), 'parameters')
flatten_parameters(model)
model.add_postmap(numericalize_postmap)
model

In [ ]:
SymbolicSharedVariable.UNEVALUATED

In [ ]:
SymbolicSharedVariable.evaluate_all_unevaluated({model['inputs'][0]:[1]})

In [ ]:
# CAUTION: build after model is complete, I am not sure why, but this gives wrong results when compiled earlier
f_model = model.function()

## target

In [ ]:
target = dm.RadialTransform(init_alpha=1, init_beta=20, init_z0=[3])

In [ ]:
f_target = target.function()
f_target([7])  # caution, needs vector not scalar input

In [ ]:
N = 40
xs = np.linspace(-10, 10, N)[:, np.newaxis]
xs[:7]

In [ ]:
ys = np.array([f_target(x) for x in xs])
ys[:7]

## visualization

In [ ]:
def plot_fit(fig, ax, time_delay=0.1):
    ax.clear() # interactive
    # target data:
    ax.plot(xs, ys, 'k:')
    
    # model:
    model_ys = np.array([f_model(x) for x in xs])
    ax.plot(xs, model_ys, 'r')
    
    fig.canvas.draw()
    time.sleep(time_delay)

In [ ]:
fig, ax = plt.subplots()
plot_fit(fig, ax)

## fit

In [ ]:
scipy_kwargs = scipy_postmap(model.postmap(wrapper=batch))
scipy_kwargs

In [ ]:
def callback(x):
    print x
    plot_fit(fig, ax)

In [ ]:
minimize(
    args=(ys,xs),
    method="Newton-CG",
    options={'maxiter':100, 'disp':True},
    callback=callback,
    **scipy_kwargs
)

In [ ]:
[p.eval() for p in model['parameters_positive']]

In [ ]:
[p.eval() for p in model['parameters']]

# two dimensions

## model

In [ ]:
model2d = dm.RadialTransform(size=2)
f_model2d = model2d.function()

## target

In [ ]:
target2d = dm.RadialTransform(init_alpha=1, init_beta=5, init_z0=[3, 1])
# target2d = dm.RadialTransform(init_alpha=50, init_beta=-40, init_z0=[3, 1])

In [ ]:
f_target2d = target2d.function()
f_target2d([7, 4])  # caution, needs vector not scalar input

In [ ]:
N2d = 10
_xs = np.linspace(-10, 10, N2d)
X1, X2 = np.meshgrid(_xs, _xs)
xs2d = np.c_[X1.flat, X2.flat]
xs2d[:5]

In [ ]:
ys2d = np.array([f_target2d(x) for x in xs2d])
ys2d[:5]

## visualization

In [ ]:
diff2d = ys2d - xs2d
quiver_kwargs = dict(scale=1, angles='xy', scale_units='xy', width=0.003)

def plot_fit2d(fig, ax, time_delay=0.1, expand_factor=0.4):
    ax.clear()
    
    # target data:
    ax.quiver(xs2d[:,0], xs2d[:,1], diff2d[:,0], diff2d[:,1], hatch='***', facecolor='none', color="k", **quiver_kwargs)
    
    # model:
    m_ys2d = np.array([f_model2d(x) for x in xs2d])
    m_diff2d = m_ys2d - xs2d
    ax.quiver(xs2d[:,0], xs2d[:,1], m_diff2d[:,0], m_diff2d[:,1], color="r", **quiver_kwargs)

    # ax.autoscale()  # does not work with arrows, do it manually:
    l, r, b, t = ax.axis()
    dx, dy = r - l, t - b
    ax.axis([l - expand_factor * dx, r + expand_factor * dx, b - expand_factor * dy, t + expand_factor * dy])
    
    fig.canvas.draw()
    time.sleep(time_delay)

In [ ]:
fig, ax = plt.subplots()
plot_fit2d(fig, ax)

## fit

In [ ]:
myoptimizer = opt.ScipyOptimizer(batch)
# CAUTION: scipy args needs to be tuple in order to work as expected!
myoptimizer.optimize(
    model2d, args=(ys2d,xs2d),
#     method="Newton-CG",
    options={'maxiter':100, 'disp':True},
    callback=lambda x: plot_fit2d(fig, ax)
)

# Probabilistic Normalizing Flow

Only the combination with gaussian base function is shown. The uniform base function has the disadvantage of not being differentiable (not yet supported).

## model

In [ ]:
size=2
base_model = pm.DiagGauss(size)
# CAUTION, this transform cannot be reused elsewhere, as it is merged into normalizing flow
# its references point always to the current substitution
transform_model = dm.RadialTransform(size, init_alpha=2)

In [ ]:
model = pm.NormalizingFlow(transform_model, base_model)
m_sampler = model.function()
m_sampler()

## target

In [ ]:
base_target = pm.DiagGauss(init_mean=[5,5], init_var=(10,5))
transform_target = dm.RadialTransform(init_z0=[5,5], init_beta=30, init_alpha=3)

In [ ]:
target = pm.NormalizingFlow(transform_target, base_target)
t_sampler = target.function()
t_sampler()

In [ ]:
n_samples = 2000
targets = np.array([t_sampler() for _ in xrange(n_samples)])

## visualization

In [ ]:
scatter_kwargs = dict(edgecolors="face", alpha=0.01, s=5000)
outer = dict(i=0)
def plot_fit(fig, ax, time_delay=0.1):
    outer['i'] += 1
    ax.clear()
    
    # targets
    ax.plot(targets[:,0], targets[:,1], 'k.')
#     ax.scatter(targets[:,0], targets[:,1], c='k', **scatter_kwargs)
    # model
    m_samples = np.array([m_sampler() for _ in xrange(n_samples)])
    ax.scatter(m_samples[:,0], m_samples[:,1], c='r',**scatter_kwargs)

    fig.canvas.draw()
    if time_delay is not None:
        time.sleep(time_delay)

In [ ]:
fig, ax = plt.subplots()
plot_fit(fig, ax)

## fit

In [ ]:
myoptimizer = opt.ScipyOptimizer(batch)
# CAUTION: scipy args needs to be tuple in order to work as expected!
myoptimizer.optimize(
    model, args=(targets,),
#     method='CG',
#     method="Newton-CG",
#     method="Nelder-Mead",
    options={'maxiter':10000, 'disp':True},
    callback=lambda x: plot_fit(fig, ax, time_delay=0.001)
)

In [ ]:
alpha = transform_model.alpha_from_Alpha(transform_model._Alpha.get_value(), module=np)
beta = transform_model.beta_from_Beta(alpha, transform_model._Beta.get_value(), module=np)
print alpha, beta